In [ ]:
import os
import shutil
from time import time

import numpy as np
import SimpleITK as sitk
import scipy.ndimage as ndimage
from pathlib import Path

import nibabel as nib
import glob
import os
import numpy as np
import cv2
import os
import json
import os, os.path
import numpy as np
import cv2
import pydicom
import glob
from scipy import ndimage, misc
import nibabel as nib
import matplotlib
import scipy.misc
from PIL import Image
import numpy as np
import imageio
import numpy as np
from skimage import io, exposure, img_as_uint, img_as_float
import pandas as pd
def hu_to_grayscale(volume, hu_min, hu_max):
    # Clip at max and min values if specified
    if hu_min is not None or hu_max is not None:
        volume = np.clip(volume, hu_min, hu_max)

    # Scale to values between 0 and 1
    mxval = np.max(volume)
    mnval = np.min(volume)
    im_volume = (volume - mnval)/max(mxval - mnval, 1e-3)

    # Return values scaled to 0-255 range, but *not cast to uint8*
    # Repeat three times to make compatible with color overlay
    im_volume = 255*im_volume
    return np.stack((im_volume, im_volume, im_volume), axis=-1)
DEFAULT_HU_MAX = 512
DEFAULT_HU_MIN = -512
DEFAULT_OVERLAY_ALPHA = 0.3

hu_min=DEFAULT_HU_MIN 
hu_max=DEFAULT_HU_MAX 
######################################### Normalization code#################################
def trueZone(imgArray):
    #return a 2d array of no-black zone boundary
    H=imgArray.shape[0]
    W=imgArray.shape[1]
   
    for h0 in range(H):
        if imgArray[h0,:].sum()!=0:
            break
    for h1 in range(H-1, -1,-1):
        if imgArray[h1,:].sum()!=0:
            break
       
    for w0 in range(W):
        if imgArray[:,w0].sum()!=0:
            break
    for w1 in range(W-1, -1,-1):
        if imgArray[:,w1].sum()!=0:
            break
    return [h0,h1, w0,w1]
######################################### Normalization code#################################
def normalization(imgArray):
    #in place function. Normalize the image
    imgFlat=imgArray.copy()
    imgFlat=np.sort(imgFlat, axis=None)

    windowlower=imgFlat[int(imgFlat.shape[0]*0.05)]
    windowupper=imgFlat[int(imgFlat.shape[0]*0.95)-1] 

    imageNormalized=np.copy(imgArray)
    imageNormalized=np.clip(imageNormalized,windowlower,windowupper)-windowlower
    imageNormalized=imageNormalized/(windowupper-windowlower)*4095
    imgArray[:,:]=imageNormalized

import operator
# center cropping
def cropND(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]


#a = np.arange(100).reshape((10,10))
#cropND(a, (5,5))
#slice: 2d np array
#a single slice of MRI
#boundray=trueZone(slice)
#imgNormalized=slice[boundray[0]:boundray[1], boundray[2]:boundray[3]]
#normalization(imgNormalized)
def cropfunction(img):
    y=int((256-128)/2)
    x=int((256-128)/2)
    # Crop, convert back from numpy to PIL Image and and save
    cropped=img[x:x+128,y:y+128]
    return cropped 

In [ ]:
path='D:\\Moonacompitionemdic\\emidec-dataset-1.0.1 (1)\\emidec-dataset-1.0.1'  # dataset path have folder tarining
# D:\Newdatasetandcodes\Maincodes2.5d\SeGThtraindata\SegTHOR2019dataset\train  in your case give the path of your system director
save_path='D:\\Moonacompitionemdic\\emidec-dataset-1.0.1 (1)\\Traintestchallenege\\Multiclassdataset\\Trainingfolder'  # save path image folder folder name is img; provid you folder name to saving image folder
#save_path1='D:\\Newdatasetandcodes\\Maincodes2.5d\\SeGThtraindata\\Datasetcolor\\mask1' # save mask path folder name should be mask; provide your folder name to saving masks folder
files=os.listdir(path)
    
for idx, volume in enumerate(files):
    total_imgs = []
    cur_file = os.path.join(path, volume)
    print(idx, cur_file)
    cur_save_path = os.path.join(save_path, volume)
    if not os.path.exists(cur_save_path):
        os.makedirs(cur_save_path)
#    cur_save_path1 = os.path.join(save_path1, volume)
#    if not os.path.exists(cur_save_path1):
#        os.makedirs(cur_save_path1)
    cur_file1=os.path.join(cur_file,'Images'+'\\'+str(volume)+'.nii.gz')
    img = nib.load(cur_file1)
    img = np.array(img.get_data())
#    img = Image.new("I", img.T.shape)
#    im = Image.fromarray(img)
#    array_buffer = img.tobytes()
#    img.frombytes(array_buffer, 'raw', "I;16")
#    im4 = exposure.rescale_intensity(img, out_range='float')
#    img = img_as_uint(im4)
    d=1
    for test in range(img.shape[2]):
        mask=img[:,:,test]
        #boundray=trueZone(mask)
        #imgNormalized=mask[boundray[0]:boundray[1], boundray[2]:boundray[3]]
        #normalization(imgNormalized)
        mask=cropND(mask, (128,128))
        #mask=cropfunction(mask)
        filename = "%d.png"%d
        cv2.imwrite(os.path.join(cur_save_path,cur_save_path+'\\'+volume+ '_' + str(test) +'_image'+ '.png'), mask) # save input images
        d+=1   

In [ ]:
#%%  saving image folder
import numpy as np
import scipy.misc  
import cv2
import numpy
import glob
import pylab as plt
import os
#import png
import pydicom
import numpy as np
from pydicom.tag import Tag
import time
from PIL import Image, ImageOps           
import numpy as np
import scipy.misc  
import cv2
import numpy
import glob
import pylab as plt
import os
from os import listdir
from os.path import isfile, join
import numpy
import cv2
from skimage import io, exposure, img_as_uint, img_as_float
#from glob import glob
#
def resize(listImg):
    #crop each image of a same patient to ensure the same image size
    xmin=9999
    ymin=9999
    for img in listImg:
        if img.shape[0]<ymin:
            ymin=img.shape[0]
        if img.shape[1]<xmin:
            xmin=img.shape[1]
            
    for _,img in enumerate(listImg):
        imgResized=img[int((img.shape[0]-ymin)/2):int((img.shape[0]-ymin)/2)+ymin,
          int((img.shape[1]-xmin)/2):int((img.shape[1]-xmin)/2)+xmin]
        if _==0:
            imgConcatenated=np.array([imgResized])
        else:
            imgConcatenated=np.concatenate((imgConcatenated, [imgResized]),axis=0)
    return np.transpose(imgConcatenated, (2,1,0))     

def trueZone(imgArray):
    #return a 2d array of no-black zone boundary
    H=imgArray.shape[0]
    W=imgArray.shape[1]
   
    for h0 in range(H):
        if imgArray[h0,:].sum()!=0:
            break
    for h1 in range(H-1, -1,-1):
        if imgArray[h1,:].sum()!=0:
            break
       
    for w0 in range(W):
        if imgArray[:,w0].sum()!=0:
            break
    for w1 in range(W-1, -1,-1):
        if imgArray[:,w1].sum()!=0:
            break
    return [h0,h1, w0,w1]

def normalization(imgArray):
    #in place function. Normalize the image
    imgFlat=imgArray.copy()
    imgFlat=np.sort(imgFlat, axis=None)

    windowlower=imgFlat[int(imgFlat.shape[0]*0.05)]
    windowupper=imgFlat[int(imgFlat.shape[0]*0.95)-1] 

    imageNormalized=np.copy(imgArray)
    imageNormalized=np.clip(imageNormalized,windowlower,windowupper)-windowlower
    imageNormalized=imageNormalized/(windowupper-windowlower)*4095
    imgArray[:,:]=imageNormalized
      
import natsort
folders = glob.glob('D:\\Moonacompitionemdic\\emidec-dataset-1.0.1 (1)\\Traintestchallenege\\Multiclassdataset\\Trainingfolder\\*')
foldername='D:\\Moonacompitionemdic\\emidec-dataset-1.0.1 (1)\\Traintestchallenege\\trainsequence'
d=1
for mypath in natsort.natsorted(folders):
    onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
    print(natsort.natsorted(onlyfiles,reverse=False))
    for n in natsort.natsorted(range(0, len(onlyfiles))):
        imge= cv2.imread( join(mypath,natsort.natsorted(onlyfiles,reverse=False)[n]) )
        #boundray=trueZone(imge)
        #imgNormalized=slice[boundray[0]:boundray[1], boundray[2]:boundray[3]]
        #imge=normalization(imgNormalized)
        #imge= cv2.resize(imge, (256,256), interpolation = cv2.INTER_AREA)
        #imge = cv2.cvtColor(imge, cv2.COLOR_BGR2GRAY)
        # y=imge
        # imge = (4096*((y - y.min())/y.ptp())).astype(np.uint8) # save using 12 bits
        # #imge= cv2.resize( imge,(256,256) )
        filename = "%d.png"%d
        #cv2.imwrite(foldername+'\\'+filename, imge) # save images
        #io.imsave(foldername+'\\'+filename, imge*255) # save images
        cv2.imwrite(foldername+'\\'+filename, imge) # save images
        d+=1        

In [ ]:
#%% Mask folder for challenege dataset 

#% cropping the mask from center and make equal size ##########################
import os
import shutil
from time import time

import numpy as np
import SimpleITK as sitk
import scipy.ndimage as ndimage
from pathlib import Path

import nibabel as nib
import glob
import os
import numpy as np
import cv2
import os
import json
import os, os.path
import numpy as np
import cv2
import pydicom
import glob
from scipy import ndimage, misc
import nibabel as nib
import matplotlib
import scipy.misc
from PIL import Image
import numpy as np
import imageio
import numpy as np
from skimage import io, exposure, img_as_uint, img_as_float
import pandas as pd
def hu_to_grayscale(volume, hu_min, hu_max):
    # Clip at max and min values if specified
    if hu_min is not None or hu_max is not None:
        volume = np.clip(volume, hu_min, hu_max)

    # Scale to values between 0 and 1
    mxval = np.max(volume)
    mnval = np.min(volume)
    im_volume = (volume - mnval)/max(mxval - mnval, 1e-3)

    # Return values scaled to 0-255 range, but *not cast to uint8*
    # Repeat three times to make compatible with color overlay
    im_volume = 255*im_volume
    return np.stack((im_volume, im_volume, im_volume), axis=-1)
DEFAULT_HU_MAX = 512
DEFAULT_HU_MIN = -512
DEFAULT_OVERLAY_ALPHA = 0.3

hu_min=DEFAULT_HU_MIN 
hu_max=DEFAULT_HU_MAX 
######################################### Normalization code#################################
def trueZone(imgArray):
    #return a 2d array of no-black zone boundary
    H=imgArray.shape[0]
    W=imgArray.shape[1]
   
    for h0 in range(H):
        if imgArray[h0,:].sum()!=0:
            break
    for h1 in range(H-1, -1,-1):
        if imgArray[h1,:].sum()!=0:
            break
       
    for w0 in range(W):
        if imgArray[:,w0].sum()!=0:
            break
    for w1 in range(W-1, -1,-1):
        if imgArray[:,w1].sum()!=0:
            break
    return [h0,h1, w0,w1]

def normalization(imgArray):
    #in place function. Normalize the image
    imgFlat=imgArray.copy()
    imgFlat=np.sort(imgFlat, axis=None)

    windowlower=imgFlat[int(imgFlat.shape[0]*0.05)]
    windowupper=imgFlat[int(imgFlat.shape[0]*0.95)-1] 

    imageNormalized=np.copy(imgArray)
    imageNormalized=np.clip(imageNormalized,windowlower,windowupper)-windowlower
    imageNormalized=imageNormalized/(windowupper-windowlower)*4095
    imgArray[:,:]=imageNormalized
import operator
# center cropping
def cropND(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]


#a = np.arange(100).reshape((10,10))
#cropND(a, (5,5))
#slice: 2d np array
#a single slice of MRI
#boundray=trueZone(slice)
#imgNormalized=slice[boundray[0]:boundray[1], boundray[2]:boundray[3]]
#normalization(imgNormalized)
def cropfunction(img):
    y=int((256-128)/2)
    x=int((256-128)/2)
    # Crop, convert back from numpy to PIL Image and and save
    cropped=img[x:x+128,y:y+128]
    return cropped

path='D:\\Moonacompitionemdic\\emidec-dataset-1.0.1 (1)\\Traintestchallenege\\Testdataset'  # dataset path have folder tarining
# D:\Newdatasetandcodes\Maincodes2.5d\SeGThtraindata\SegTHOR2019dataset\train  in your case give the path of your system director
save_path='D:\\Moonacompitionemdic\\emidec-dataset-1.0.1 (1)\\Traintestchallenege\\Testdatafolder\\mmask'  # save path image folder folder name is img; provid you folder name to saving image folder
#save_path1='D:\\Newdatasetandcodes\\Maincodes2.5d\\SeGThtraindata\\Datasetcolor\\mask1' # save mask path folder name should be mask; provide your folder name to saving masks folder
files=os.listdir(path)
    
for idx, volume in enumerate(files):
    total_imgs = []
    cur_file = os.path.join(path, volume)
    print(idx, cur_file)
    cur_save_path = os.path.join(save_path, volume)
    if not os.path.exists(cur_save_path):
        os.makedirs(cur_save_path)
#    cur_save_path1 = os.path.join(save_path1, volume)
#    if not os.path.exists(cur_save_path1):
#        os.makedirs(cur_save_path1)
    cur_file1=os.path.join(cur_file,'Contours'+'\\'+str(volume)+'.nii.gz')
    img = nib.load(cur_file1)
    img = np.array(img.get_data())
#    img = Image.new("I", img.T.shape)
#    im = Image.fromarray(img)
#    array_buffer = img.tobytes()
#    img.frombytes(array_buffer, 'raw', "I;16")
#    im4 = exposure.rescale_intensity(img, out_range='float')
#    img = img_as_uint(im4)
    d=1
    for test in range(img.shape[2]):
        mask=img[:,:,test]
#        boundray=trueZone(mask)
#        imgNormalized=mask[boundray[0]:boundray[1], boundray[2]:boundray[3]]
#        normalization(imgNormalized)
        imge=cropND(mask, (128,128))
        # imge[imge==2]=2
        # imge[imge==1]=0
        # imge[imge==3]=1
        # imge[imge==4]=1
        # imge[imge>=1]=1
        #mask=cropfunction(mask)
        filename = "%d.png"%d
        io.imsave(os.path.join(cur_save_path,cur_save_path+'\\'+volume + '_' + str(test) +'_mask'+ '.png'), imge*255) # save input images
        d+=1
#%

In [ ]:
#%%  saving image folder
import numpy as np
import scipy.misc  
import cv2
import numpy
import glob
import pylab as plt
import os
#import png
import pydicom
import numpy as np
from pydicom.tag import Tag
import time
from PIL import Image, ImageOps           
import numpy as np
import scipy.misc  
import cv2
import numpy
import glob
import pylab as plt
import os
from os import listdir
from os.path import isfile, join
import numpy
import cv2
from skimage import io, exposure, img_as_uint, img_as_float
#from glob import glob
#
def resize(listImg):
    #crop each image of a same patient to ensure the same image size
    xmin=9999
    ymin=9999
    for img in listImg:
        if img.shape[0]<ymin:
            ymin=img.shape[0]
        if img.shape[1]<xmin:
            xmin=img.shape[1]
            
    for _,img in enumerate(listImg):
        imgResized=img[int((img.shape[0]-ymin)/2):int((img.shape[0]-ymin)/2)+ymin,
          int((img.shape[1]-xmin)/2):int((img.shape[1]-xmin)/2)+xmin]
        if _==0:
            imgConcatenated=np.array([imgResized])
        else:
            imgConcatenated=np.concatenate((imgConcatenated, [imgResized]),axis=0)
    return np.transpose(imgConcatenated, (2,1,0))     

def trueZone(imgArray):
    #return a 2d array of no-black zone boundary
    H=imgArray.shape[0]
    W=imgArray.shape[1]
   
    for h0 in range(H):
        if imgArray[h0,:].sum()!=0:
            break
    for h1 in range(H-1, -1,-1):
        if imgArray[h1,:].sum()!=0:
            break
       
    for w0 in range(W):
        if imgArray[:,w0].sum()!=0:
            break
    for w1 in range(W-1, -1,-1):
        if imgArray[:,w1].sum()!=0:
            break
    return [h0,h1, w0,w1]

def normalization(imgArray):
    #in place function. Normalize the image
    imgFlat=imgArray.copy()
    imgFlat=np.sort(imgFlat, axis=None)

    windowlower=imgFlat[int(imgFlat.shape[0]*0.05)]
    windowupper=imgFlat[int(imgFlat.shape[0]*0.95)-1] 

    imageNormalized=np.copy(imgArray)
    imageNormalized=np.clip(imageNormalized,windowlower,windowupper)-windowlower
    imageNormalized=imageNormalized/(windowupper-windowlower)*4095
    imgArray[:,:]=imageNormalized
      
import natsort
folders = glob.glob('D:\\Moonacompitionemdic\\emidec-dataset-1.0.1 (1)\\Traintestchallenege\\Testdatafolder\\mmask\\*')
foldername='D:\\Moonacompitionemdic\\emidec-dataset-1.0.1 (1)\\Traintestchallenege\\masksequence'
d=1
for mypath in natsort.natsorted(folders):
    onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
    print(natsort.natsorted(onlyfiles,reverse=False))
    for n in natsort.natsorted(range(0, len(onlyfiles))):
        imge= cv2.imread( join(mypath,natsort.natsorted(onlyfiles,reverse=False)[n]) )
        #boundray=trueZone(imge)
        #imgNormalized=slice[boundray[0]:boundray[1], boundray[2]:boundray[3]]
        #imge=normalization(imgNormalized)
        #imge= cv2.resize(imge, (256,256), interpolation = cv2.INTER_AREA)
        #imge = cv2.cvtColor(imge, cv2.COLOR_BGR2GRAY)
        # y=imge
        # imge = (4096*((y - y.min())/y.ptp())).astype(np.uint8) # save using 12 bits
        # #imge= cv2.resize( imge,(256,256) )
        filename = "%d.png"%d
        #cv2.imwrite(foldername+'\\'+filename, imge) # save images
        #io.imsave(foldername+'\\'+filename, imge*255) # save images
        cv2.imwrite(foldername+'\\'+filename, imge) # save images
        d+=1        